In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import scipy as sp
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Textarea
from iam_utils import *
import ase
from ase.io import read, write
from ase.calculators import lj, eam
from tqdm.notebook import tqdm

In [ ]:
#### AVOID folding of output cell 

In [ ]:
%%html

<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:5000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [ ]:
data_dump = WidgetDataDumper(prefix="module_07")
display(data_dump)

In [ ]:
module_summary = Textarea("general comments on this module", layout=Layout(width="100%"))
data_dump.register_field("module-summary", module_summary, "value")
display(module_summary)

_Reference textbook: ???? _

# Data-driven modeling 

This module provides a very brief and over-simplified primer on "data-driven" modeling. 
In abstract terms, a data-driven approach attempts to establish a relationship between _input_ data and _target_ properties (or to recognize patterns in the data itself) without using deductive reasoning, i.e. without proceeding though a series of logical steps starting from an hypothesis on the physical behavior of a system. 

Instead, the empirical association between inputs and targets is taken as the only basis to establish an _inductive_ relationship between them. The traditional scientific method proceeds through a combination of induction and deduction, while data-driven approaches are intended to be entirely inductive. On the risks of purely inductive reasoning, see [Bertrand Russel's inductivist chicken story](http://www.ditext.com/russell/rus6.html). In practice, _inductive biases_ are often included in the modeling, through by means of the choices that are made in the construction and the tuning of the model itself: this is how a component of physics-inspired (deductive) concepts can make it back into machine learning. 

As the most primitive example of data-driven modeling, consider the case of _linear regression_. 
A set of $n_\mathrm{train}$ data points and targets $\{x_i, y_i\} $ are assumed to follow a linear relationship of the form $y(x)=a x$, where the slope $a$ is an adjustable parameter. 
For a given value of $a$, one can compute the _loss_, i.e. the root mean square error between the true value of the targets and the predictions of the model,

$$
\ell = \frac{1}{n_\mathrm{train}} \sum_i |y(x_i)-y_i|^2 
$$

This widget allows you to play around with the core idea of linear regression: by adjusting the value of $a$ you can minimize the discrepancy between predictions and targets, and find the best model within the class chosen to represent the input-target relationship

In [ ]:
np.random.seed(1234)
lr_x = (np.random.uniform(size=20)-0.5)*10
lr_y = 2.33*lr_x+(np.random.uniform(size=20)-0.5)*2
def lr_plot(ax, a):    
    ax.plot(lr_x, lr_y, 'b.')
    ax.plot([-5,5],[-5*a,5*a], 'r--')
    l = np.mean((lr_y-a*lr_x)**2)
    ax.text(-4,8,f'$\ell = ${l:.3f}')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
wp_lr = WidgetPlot(lr_plot, WidgetParbox(a=(1.0, -5.0, 5.0, 0.1, r'$a$')))
display(wp_lr)

<span style="color:blue">**01** What is (roughly) the best value of $a$ that minimizes the loss in the linear regression model? </span>

In [ ]:
ex1_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex1-answer", ex1_txt, "value")
display(ex1_txt)

In a linear regression model, the loss can be minimized with a closed expression: given that $\partial \ell/\partial a \propto \sum_i x_i(a x_i -y_i)$, it's easy to see that an extremum occurs for $a = \sum_i x_i y_i / \sum_i x_i^2.$

In [ ]:
lr_x@lr_y/

# Fingerprints and descriptors

# Dimensionality reduction

# Regression